In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')
sys.path.append('../')

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import editdistance
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


In [ ]:
data=pd.read_csv("../simulations/evaluations3/consistency_robustness_independence/Greedy_mu1_tr0.05_r0.1_rho1.csv",index_col=False)
data.head()

In [ ]:
data.landscape_id.unique()

In [ ]:
sub_data=data[(data.model_type.str.startswith("NAMb_ss1"))&\
              (data.landscape_id=='B1L14RNA1')]

In [ ]:
sub_data[["true_score","model_score"]].corr("pearson")

In [ ]:
plt.scatter(sub_data["true_score"],sub_data["model_score"])

In [ ]:
plt.figure(figsize=(14,5))
sns.stripplot(x="measurement_cost",y="true_score",hue="model_type",data=sub_data)

In [ ]:
data["start_id"].unique()

In [ ]:
import timeit

In [ ]:
def get_top_sequence_per_batch(data):
    top_per_batch=[]
    for i in data.batch.unique():
        sub_data=data[data.batch==i] #must also group by model type and landscape id
        #print (sub_data.true_score.max())
        top_per_batch.append(sub_data.true_score.max())
    return (top_per_batch)

In [ ]:
outcomes=pd.DataFrame(columns=["landscape_id","start_id","model_type","batch","max_fitness","cum_max_fitness"])
for landscape in data.landscape_id.unique():
    if landscape==str(-1) or start_id==str(-1):
        continue
    print(landscape)
    valid_start_ids=[x for x in data.start_id.unique() if landscape[landscape.index("L"): landscape.index("R")] in x]
    for start_id in valid_start_ids:
        for noisy_model in data.model_type.unique():
            sub_data = data[(data.landscape_id==landscape)&\
                           (data.start_id==start_id)&\
                           (data.model_type==noisy_model)] 
            cum_fit=0
            for batch, max_fitness in enumerate(get_top_sequence_per_batch(sub_data)):
                if max_fitness>cum_fit:
                   cum_fit = max_fitness 
                outcomes = outcomes.append(pd.DataFrame.from_records([{"landscape_id":landscape,\
                                      "start_id":start_id,\
                                      "model_type": noisy_model,\
                                      "batch": batch+1, \
                                      "max_fitness": max_fitness,\
                                      "cum_max_fitness": cum_fit}])) 

In [ ]:
outcomes.head()

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(x="batch",y="cum_max_fitness",hue="model_type",data= outcomes[outcomes.landscape_id=="B1L50RNA1"])